In [1]:
import torch
import random
import numpy as np
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

class PasswordNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PasswordNN, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 7)
        self.hidden2 = nn.Linear(7, 7)
        self.hidden3 = nn.Linear(7, 5)
        self.output = nn.Linear(5, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = self.output(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device selected: {device}')

model = PasswordNN(input_dim=11, output_dim=3).to(device)
model.load_state_dict(torch.load("PasswordNN.pth"))
model.eval()
print("Model loaded successfully.")

import joblib
scaler = joblib.load("scaler.pkl")
print('Selected scalar loaded!')


Device selected: cuda
Model loaded successfully.
Selected scalar loaded!


/tmp/ipykernel_22730/4291526772.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PasswordNN.pth"))


In [2]:
# Sequential Pattern Detection
def detect_sequential_pattern(password):
    sequential_patterns = [
        "abcdefghijklmnopqrstuvwxyz", 
        "ABCDEFGHIJKLMNOPQRSTUVWXYZ", 
        "0123456789", 
        "qwertyuiop", "asdfghjkl", "zxcvbnm"
    ]
    
    for seq in sequential_patterns:
        for i in range(len(seq) - 2):  # Look for at least 3-character sequences
            if seq[i:i+3] in password:
                return 1
    return 0

# Repeated Characters Detection
def detect_repeated_characters(password):
    max_repeated = 0
    current_count = 1
    
    for i in range(1, len(password)):
        if password[i] == password[i - 1]:
            current_count += 1
            max_repeated = max(max_repeated, current_count)
        else:
            current_count = 1
    
    return max_repeated

# Preprocessing the Input
def preprocess_input(user_input):
    # Randomized `count` value
    count = random.randint(10, 15000)
    
    # Feature Extraction
    length = len(user_input)
    common = 1 if user_input.lower() in ['password', '123456', 'qwerty'] else 0
    alpha_count = sum(c.isalpha() for c in user_input)
    numeric_count = sum(c.isdigit() for c in user_input)
    special_count = sum(not c.isalnum() for c in user_input)
    uppercase = sum(c.isupper() for c in user_input)
    lowercase = sum(c.islower() for c in user_input)
    sequential_pattern = detect_sequential_pattern(user_input)
    repeated_characters = detect_repeated_characters(user_input)
    case_ratio = uppercase / (lowercase + 1e-5)  # Prevent division by zero

    # Feature Vector and Scaling
    feature_vector = np.array([
        count, length, common, alpha_count, numeric_count, special_count, 
        uppercase, lowercase, sequential_pattern, repeated_characters, case_ratio
    ])
    feature_vector_scaled = scaler.transform(feature_vector.reshape(1, -1))  # Assumes `scaler` is fitted
    
    # Convert to Tensor
    input_tensor = torch.tensor(feature_vector_scaled, dtype=torch.float32).to(device)
    return input_tensor

# Prediction Function
def test_model_with_input(user_input):
    input_tensor = preprocess_input(user_input)
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted_class = torch.max(output, 1)
        class_names = ['weak', 'moderate', 'strong']
        predicted_label = class_names[predicted_class.item()]
        print(f"Predicted password strength: {predicted_label}")

In [ ]:
password = "123"
test_model_with_input(password)

Predicted password strength: weak
